This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2021-12-02 21:52:53--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘www.dropbox.com’


1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [1]:
import pandas as pd 
import numpy as np
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)[:10000]
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
9995,1,0,61,3869200267,696,11
9996,1,0,60,3869200267,701,5
9997,1,0,59,3869200267,707,23
9998,1,0,63,3869200267,706,15


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [24]:
a=data.groupby('ORBIT_CNT').max()
b=data.groupby('ORBIT_CNT').min()
x=input()
data.iloc[int(x)]
print(df.at[dates[1], 'A'])

5


HEAD                    1
FPGA                    0
TDC_CHANNEL            63
ORBIT_CNT      3869200167
BX_COUNTER           2762
TDC_MEAS                4
Name: 5, dtype: int64

3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [54]:
data['Duration'] = pd.Timestamp('200213')


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [31]:
data['Duration'] = pd.datetime.now()
data

/tmp/ipykernel_3337/2121500647.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  data['Duration'] = pd.datetime.now()


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Duration
0,1,0,123,3869200167,2374,26,2021-12-03 19:33:34.313326
1,1,0,124,3869200167,2374,27,2021-12-03 19:33:34.313326
2,1,0,63,3869200167,2553,28,2021-12-03 19:33:34.313326
3,1,0,64,3869200167,2558,19,2021-12-03 19:33:34.313326
4,1,0,64,3869200167,2760,25,2021-12-03 19:33:34.313326
...,...,...,...,...,...,...,...
9995,1,0,61,3869200267,696,11,2021-12-03 19:33:34.313326
9996,1,0,60,3869200267,701,5,2021-12-03 19:33:34.313326
9997,1,0,59,3869200267,707,23,2021-12-03 19:33:34.313326
9998,1,0,63,3869200267,706,15,2021-12-03 19:33:34.313326


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [52]:
data.groupby('TDC_CHANNEL').max()[-3:]

,HEAD,FPGA,ORBIT_CNT,BX_COUNTER,TDC_MEAS,Duration
TDC_CHANNEL,,,,,,
127,1,1,3869200247,3337,29,2013-01-02
128,1,1,3869200253,3337,23,2013-01-02
139,1,1,3869200267,3559,0,2013-01-02


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [57]:
data[data['TDC_CHANNEL']==139].count()

HEAD           860
FPGA           860
TDC_CHANNEL    860
ORBIT_CNT      860
BX_COUNTER     860
TDC_MEAS       860
Duration       860
dtype: int64

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel